In [ ]:
# Install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Install spark (change the version number if needed)


!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

In [ ]:
#Unzip the spark file to the current folder


!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [ ]:
#Set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [ ]:
import csv
import pandas as pd

In [ ]:
#Install findspark using pip
!pip install -q findspark

ERROR: Operation cancelled by user


In [ ]:
pip install pyspark==3.0.0

In [ ]:
#Spark for Python (pyspark)
# !pip install pyspark


In [ ]:
#importing pyspark
# import pyspark
from pyspark import SparkContext, SparkConf
import pandas as pd 


In [ ]:
#importing sparksessio
from pyspark.sql import SparkSession
import gc
gc.enable()

In [ ]:
import findspark
findspark.init('/content/spark-3.0.0-bin-hadoop3.2')
# from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').config('spark.sql.execution.arrow.pyspark.enabled', True).config('spark.sql.session.timeZone', 'UTC').config('spark.driver.memory','40G').config('spark.ui.showConsoleProgress', True).config('spark.sql.repl.eagerEval.enabled', True).getOrCreate()


In [ ]:
 sc = SparkContext.getOrCreate();


In [ ]:
#printing the version of spark
print("Apache Spark version: ", spark.version)

Apache Spark version:  3.0.0


In [ ]:
order_products_prior = spark.read.csv("/content/drive/MyDrive/Instacart/order_products__prior.csv",header=True)
# order_products_prior.show(5)

In [ ]:
orders = spark.read.csv("/content/drive/MyDrive/Instacart/orders.csv",header=True)
# orders = spark.read.csv("/content/DivisionStrength.txt",sep=",",header=True)
# orders.show(12)

In [ ]:
ordersJoined = orders.join(order_products_prior, orders.order_id == order_products_prior.order_id,'inner')
# ordersJoined.show()


In [ ]:
del orders,order_products_prior
gc.collect()

496

In [ ]:
aisles = spark.read.csv("/content/drive/MyDrive/Instacart/aisles.csv",header=True)
aisles.show(5)

In [ ]:
## Get total Number of Aisles : Working

In [ ]:
getCount=aisles.rdd.map(lambda x: (None,1))

In [ ]:
countReduced=getCount.reduceByKey(lambda a,b:a+b)

In [ ]:
aislesCollected=countReduced.collect()
aislesCollected=aislesCollected[0][1]
aislesCollected

In [ ]:
## Total Orders per userId

In [ ]:
getOrdersPerUsed=order_products_prior.rdd.map(lambda x: (x[1],1))

In [ ]:
ordersPerUserReduced=getOrdersPerUsed.reduceByKey(lambda a,b:a+b)

In [ ]:
ordersPerUserCollected=ordersPerUserReduced.collect()

In [ ]:
## total Reorders per userid

In [ ]:
getReordersPerUser=order_products_prior.rdd.map(lambda x: (x[1],int(x['reordered'])))
# getTotalReordered.collect()

In [ ]:
getTotalReorderedReduced=getReordersPerUser.reduceByKey(lambda a,b:a+b,4)

In [ ]:
getTotalReorderedReducedCol=getTotalReorderedReduced.collect()
getTotalReorderedReducedCol

In [ ]:
## Max order number be user id on Joined Table : Done as csv

In [ ]:
maxOrderNumberByUserId=ordersJoined.rdd.map(lambda x:(x['user_id'],int(x['order_number'])))

In [ ]:
maxOrderByUserReduced=maxOrderNumberByUserId.reduceByKey(lambda a,b:max(a,b))

In [ ]:
getMaxOrderReducedCol=maxOrderByUserReduced.collect()
getMaxOrderReducedCol

In [ ]:
## Get Mean of Reorder Per UserId : Done as csv

In [ ]:
reOrderByUserId=ordersJoined.rdd.map(lambda x:(x['user_id'],int(x['reordered'])))

In [ ]:
reOrderByUserAvg=reOrderByUserId.groupByKey().mapValues(lambda x: sum(x)/len(x)).collect()
reOrderByUserAvg

[('71915', 0.6602564102564102),
 ('105350', 0.6956521739130435),
 ('115070', 0.29069767441860467),
 ('49540', 0.3611111111111111),
 ('180206', 0.45722713864306785),
 ('123063', 0.7032967032967034),
 ('97416', 0.5378486055776892),
 ('203633', 0.8431876606683805),
 ('69279', 0.49221183800623053),
 ('160185', 0.6544715447154471),
 ('205105', 0.37777777777777777),
 ('53805', 0.791005291005291),
 ('61793', 0.20270270270270271),
 ('122166', 0.5952755905511811),
 ('126806', 0.5),
 ('98158', 0.7606112054329371),
 ('123823', 0.5540983606557377),
 ('154763', 0.3557692307692308),
 ('61399', 0.6858080393765381),
 ('157610', 0.6089385474860335),
 ('5603', 0.5106382978723404),
 ('86802', 0.8299445471349353),
 ('33423', 0.6206896551724138),
 ('157976', 0.5952380952380952),
 ('27853', 0.717391304347826),
 ('93434', 0.06896551724137931),
 ('114336', 0.42592592592592593),
 ('67269', 0.6805555555555556),
 ('143439', 0.5073529411764706),
 ('101393', 0.6266233766233766),
 ('75517', 0.746938775510204),
 ('1

In [ ]:
df = pd.DataFrame(reOrderByUserAvg)
df.to_csv('AverageReorderPerUser.csv')

In [ ]:
## Avg days since prior order by user id : Not Done
# daysPriorOrderByUserId=ordersJoined.rdd.map(lambda x:(x['user_id'],int(x['days_since_prior_order'])))

In [ ]:
# priordaysByUserAvg=daysPriorOrderByUserId.groupByKey().mapValues(lambda x: sum(x)/len(x)).collect()
# priordaysByUserAvg

In [ ]:
# df = pd.DataFrame(priordaysByUserAvg)
# df.to_csv('AvgDaysSincePriorOrderPerUser.csv')

In [ ]:
## Mean of Reordered By product Id : Done as csv
meanReorderByProdId=ordersJoined.rdd.map(lambda x:(x['product_id'],int(x['reordered'])))

In [ ]:
reorderedAvgByProd=meanReorderByProdId.groupByKey().mapValues(lambda x: sum(x)/len(x)).collect()
reorderedAvgByProd

[('24799', 0.6889538088413626),
 ('5785', 0.8057024603357094),
 ('36865', 0.74659583562313),
 ('22947', 0.6894977168949772),
 ('248', 0.4002511379689217),
 ('27999', 0.3266781411359725),
 ('3717', 0.5831381733021077),
 ('4137', 0.5383235485975212),
 ('34429', 0.6845064960248206),
 ('23236', 0.44865054328776727),
 ('24024', 0.8126624973649076),
 ('5652', 0.41754385964912283),
 ('31589', 0.3345724907063197),
 ('34092', 0.24649714582252205),
 ('47570', 0.5617183251767265),
 ('45806', 0.558641975308642),
 ('34458', 0.5875044373446929),
 ('11421', 0.455985095482068),
 ('820', 0.5620967741935484),
 ('11951', 0.6521739130434783),
 ('21953', 0.6499631540162122),
 ('39350', 0.44144144144144143),
 ('39101', 0.751358695652174),
 ('1618', 0.6020114942528736),
 ('29315', 0.5616724738675958),
 ('31713', 0.5019607843137255),
 ('1938', 0.3363136176066025),
 ('11760', 0.6291834002677377),
 ('8730', 0.56282722513089),
 ('35769', 0.12797881729920565),
 ('41249', 0.3134715025906736),
 ('17617', 0.59966777

In [ ]:
df = pd.DataFrame(reorderedAvgByProd)
df.to_csv('AvgReorderedPerProdId.csv')

In [ ]:
## Group BY product id get count of orders : Done as csv

In [ ]:
ordersCountPerProductId=ordersJoined.rdd.map(lambda x:(x['product_id'],1))

In [ ]:
ordersCountPerProductIdReduced=ordersCountPerProductId.reduceByKey(lambda a,b:a+b,4)

In [ ]:
ordersCountPerProductIdCollected=ordersCountPerProductIdReduced.collect()

In [ ]:
# ordersCountPerProductIdCollected

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(ordersCountPerProductIdCollected)
df.to_csv('ordersCountPerProductIdCollected.csv')

In [ ]:
#### JOIN ATTEMPT #####

In [ ]:
ordersCol=orders.columns
ordersCol

In [ ]:
ordersProdCol=order_products_prior.columns
ordersProdCol

In [ ]:
newCol=ordersCol[1:]+ordersProdCol[1:]
newCol

In [ ]:
rdd3=order_products_prior.rdd.map(lambda x: (x[0],("op",x[1:])))

In [ ]:
rdd2=orders.rdd.map(lambda x: (x[0],("o",x[1:])))

In [ ]:
rddNew=rdd3.union(rdd2)

In [ ]:
# Shuffle and sort step
# grouped=rddNew.groupByKey()
# grouped.take(1)

In [ ]:
reduced1=rddNew.reduceByKey(lambda a,b:a[1]+b[1] if a[0]=="o" else b[1]+a[1])
# reduced1.take(1)
# for element in reduced.collect():
#     print(element)`

In [ ]:
mapper0=reduced1.map(lambda x: x[0])
# mapper[0]

In [ ]:
mapper1=reduced1.map(lambda x:x[1]).collect()
# mapper1

In [ ]:
df = spark.createDataFrame(
      [mapper1[i] for i in range(len(mapper1))],
    newCol  # add your column names here
)
df.show()